# Mission: Kill a Kubernetes POD with Reliably and the Chaos Toolkit.

You can learn from experiencing unexpected events in your operationl Environment. 

In this mission  you are going to kill a POD in a Kubernetes environment (the unexpected event) and see if there are any discernable impacts to your environment.



## Using a Kubernetes Cluster

To use Kubernetes you will need some tools installed in your environment. To use Kubernetes you will need to install ```kubectl``` this is detailed in the [Kubernetes Documentation](https://kubernetes.io/docs/tasks/tools/). 

You will also need a Cluster where you can practice your reliability exercises, ideally, you will already have a Cluster running a system that is already well known to you. If you already have a cluster you can jump directly to the [Disrupting Kubernetes PODs](#disrupting-kubernetes-pods-with-the-chaos-toolkit) section.

If you don't have a cluster that you can use the following section will go through the steps to set up a Cluster with [Minkube](https://kubernetes.io/docs/setup/learning-environment/minikube/).

## Setup a Cluster with Minikube

Minikube is a tool that lets you run Kubernetes locally. Minikube runs a single-node Kubernetes cluster on your personal computer (including Windows, macOS and Linux PCs)to get started you will need to [install minikube](https://kubernetes.io/docs/tasks/tools/install-minikube/).

Once minikube is installed you can run:

```
minikube start
```
You should see output similar to:

![Minikube Start Screenshot](minikube_start_screenshot.png)

## Create a System under Test

you will need a system to test with an exposed endpoint that you can use to determine the availability of your system. As an example you could a simple Nginx hello world web server:

```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx
  labels:
    app: nginx
spec:
  selector:
    matchLabels:
      app: nginx
  replicas: 1
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginxdemos/hello
        ports:
        - containerPort: 80
```

This can be deployed to your cluster with:

```
kubectl apply -f nginx.yaml --namespace=chaos-testing
deployment.apps/nginx created
```

To connect to the webserver you will need to expose a service:

```
kubectl expose deployment nginx --type="LoadBalancer"
```

Then you can connect to the system using your default browser using minkube:

```
minikube service nginx
```
![Minikube Service](minikube-service-screenshot.png)

Your default browser should open showing:

![Nginx Screenshot](nginx-hello-screenshot.png)







## Disrupting Kubernetes PODs with the Chaos Toolkit

Now you have a cluster up and running you can disrupt the cluster by performing an action like killing a POD, using a tool such as the Chaos Toolkit. You are now going to install the Chaos Toolkit in this notebook and then use it to impact your cluster.

In [5]:
%pip install chaostoolkit

  Using cached chaostoolkit_lib-1.15.0-py3-none-any.whl (56 kB)
  Attempting uninstall: chaostoolkit-lib
    Found existing installation: chaostoolkit-lib 1.12.0
    Uninstalling chaostoolkit-lib-1.12.0:
      Successfully uninstalled chaostoolkit-lib-1.12.0
You should consider upgrading via the &#39;/Users/grant/.venvs/vs-code/bin/python -m pip install --upgrade pip&#39; command.
Note: you may need to restart the kernel to use updated packages.


You can verify the command by running:

In [6]:
!chaos --version

chaos, version 1.7.0


As you are using kubernetes as a target system you are now going to install the Chaos Toolkit [kubernetes extension](https://docs.chaostoolkit.org/drivers/kubernetes/):

In [8]:
%pip install chaostoolkit-kubernetes

  Using cached chaostoolkit_kubernetes-0.23.0-py3-none-any.whl (31 kB)
Note: you may need to restart the kernel to use updated packages.


### Confirm the System can Survive POD Death

Now you have the ```chaostoolkit``` installed along with the ```chaostoolkit-kubernetes``` extension you can use them to apply some disruption on your Kubernetes system.  Here you are going to run an experiment that will kill a POD. In this case, it will be a random POD, but if you have set up the simple system under test as described above there is only one POD, so that is the one to be killed.


In [24]:
!chaos run kill-pod-experiment.json

[2020-10-02 15:34:50 INFO] Validating the experiment&#39;s syntax
[2020-10-02 15:34:50 INFO] Experiment looks valid
[2020-10-02 15:34:50 INFO] Running experiment: Do we remain available in face of pod going down?
[2020-10-02 15:34:50 INFO] Steady-state strategy: default
[2020-10-02 15:34:50 INFO] Rollbacks strategy: default
[2020-10-02 15:34:50 INFO] Steady state hypothesis: Application is normal
[2020-10-02 15:34:50 INFO] Probe: application-must-respond-normally
[2020-10-02 15:34:50 INFO] Steady state hypothesis is met!
[2020-10-02 15:34:50 INFO] Playing your experiment&#39;s method now...
[2020-10-02 15:34:50 INFO] Action: terminate-pod
[2020-10-02 15:34:51 INFO] Pausing after activity for 5s...
[2020-10-02 15:34:56 INFO] Steady state hypothesis: Application is normal
[2020-10-02 15:34:56 INFO] Probe: application-must-respond-normally
[2020-10-02 15:34:56 INFO] Steady state hypothesis is met!
[2020-10-02 15:34:56 INFO] Let&#39;s rollback...
[2020-10-02 15:34:56 INFO] No declared roll

## Viewing the impact

You will observe from the Steady State Hypothesis step shown in the output log that the Hypothesis step shows the application responds normally before and after the terminte pod action step. This is because by default kubernetes will restart the POD when it is killed, so it's hard to observe the impact of killing the POD if you are using the simnple example above but if you use a command like the following:

```
watch -n 1 curl -I http://127.0.0.1:55982/
```

Where http://127.0.0.1:55982/ is the address given by the ```minikube service nginx``` command above.